## P7 - Prep the data 

---

In [1]:
# fmt: off

import warnings

import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

warnings.filterwarnings('ignore')


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

---

In [3]:
# Fonction Cleaning Finale
def cleaning(features, test_features, encoding="ohe"):
    """Fonction cleaning finale"""
    # Extract the ids
    train_ids = features["SK_ID_CURR"]
    test_ids = test_features["SK_ID_CURR"]

    # Extract the labels for training
    labels = features["TARGET"]

    # Remove the ids and target
    features = features.drop(columns=["SK_ID_CURR", "TARGET"])
    test_features = test_features.drop(columns=["SK_ID_CURR"])

    # One Hot Encoding
    if encoding == "ohe":
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)

        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join="inner", axis=1)

        # No categorical indices to record
        cat_indices = "auto"

    # Integer label encoding
    elif encoding == "le":
        # Create a label encoder
        label_encoder = LabelEncoder()

        # List for storing categorical indices
        cat_indices = []

        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == "object":
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(
                    np.array(features[col].astype(str)).reshape((-1,))
                )
                test_features[col] = label_encoder.transform(
                    np.array(test_features[col].astype(str)).reshape((-1,))
                )

                # Record the categorical indices
                cat_indices.append(i)

    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")

    print("Training Data Shape: ", features.shape)
    print("Testing Data Shape: ", test_features.shape)

    # Extract feature names
    feature_names = list(features.columns)

    # Impute the domainnomial features
    imputer = SimpleImputer(strategy="median")

    features = imputer.fit_transform(features)
    test_features = imputer.transform(test_features)

    # Scale the domainnomial features
    scaler = MinMaxScaler(feature_range=(0, 1))

    features = scaler.fit_transform(features)
    test_features = scaler.transform(test_features)

    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)

    return feature_names, labels, features, test_features, train_ids, test_ids


In [4]:
# Training data
app_train = pd.read_csv("../../data/raw/application_train.csv")
print("Training data shape: ", app_train.shape)
app_train.head(2)

# Testing data features
app_test = pd.read_csv("../../data/raw/application_test.csv")
print("Testing data shape: ", app_test.shape)
app_test.head(2)

# copy to add fe
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()

# feature engineering with domain knowledge variables
app_train_domain["CREDIT_INCOME_PERCENT"] = (
    app_train_domain["AMT_CREDIT"] / app_train_domain["AMT_INCOME_TOTAL"]
)
app_train_domain["ANNUITY_INCOME_PERCENT"] = (
    app_train_domain["AMT_ANNUITY"] / app_train_domain["AMT_INCOME_TOTAL"]
)
app_train_domain["CREDIT_TERM"] = (
    app_train_domain["AMT_ANNUITY"] / app_train_domain["AMT_CREDIT"]
)
app_train_domain["DAYS_EMPLOYED_PERCENT"] = (
    app_train_domain["DAYS_EMPLOYED"] / app_train_domain["DAYS_BIRTH"]
)

app_test_domain["CREDIT_INCOME_PERCENT"] = (
    app_test_domain["AMT_CREDIT"] / app_test_domain["AMT_INCOME_TOTAL"]
)
app_test_domain["ANNUITY_INCOME_PERCENT"] = (
    app_test_domain["AMT_ANNUITY"] / app_test_domain["AMT_INCOME_TOTAL"]
)
app_test_domain["CREDIT_TERM"] = (
    app_test_domain["AMT_ANNUITY"] / app_test_domain["AMT_CREDIT"]
)
app_test_domain["DAYS_EMPLOYED_PERCENT"] = (
    app_test_domain["DAYS_EMPLOYED"] / app_test_domain["DAYS_BIRTH"]
)

# Create an anomalous flag column
app_train_domain["DAYS_EMPLOYED_ANOM"] = app_train_domain["DAYS_EMPLOYED"] == 365243
# Replace the anomalous values with nan
app_train_domain["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)

app_test_domain["DAYS_EMPLOYED_ANOM"] = app_test_domain["DAYS_EMPLOYED"] == 365243
app_test_domain["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)


app_train_domain["DAYS_EMPLOYED_ANOM"] = app_train_domain["DAYS_EMPLOYED_ANOM"].astype(
    "int32"
)
app_test_domain["DAYS_EMPLOYED_ANOM"] = app_test_domain["DAYS_EMPLOYED_ANOM"].astype(
    "int32"
)

Training data shape:  (307511, 122)
Testing data shape:  (48744, 121)


In [ ]:
# if "TARGET" in app_train_domain.columns.tolist():
#    app_train_domain = app_train_domain.drop(columns="TARGET")

# domain_features_names = list(app_train_domain.columns)

# Impute the domainnomial features
# imputer = SimpleImputer(strategy="median")

# domain_features = imputer.fit_transform(app_train_domain)
# domain_features_test = imputer.transform(app_test_domain)

# Scale the domainnomial features
# scaler = MinMaxScaler(feature_range=(0, 1))

# domain_features = scaler.fit_transform(domain_features)
# domain_features_test = scaler.transform(domain_features_test)


In [5]:
liste_features, y_train, X_train, X_test, train_ids, test_ids = cleaning(
    app_train_domain, app_test_domain, encoding="ohe"
)
train_final = pd.DataFrame(X_train, columns=liste_features)
train_final["LABELS"] = y_train
train_final["SK_ID_CURR"] = train_ids

test_final = pd.DataFrame(X_test, columns=liste_features)
test_final["SK_ID_CURR"] = test_ids


Training Data Shape:  (307511, 246)
Testing Data Shape:  (48744, 246)


---

## Models

### Models scores

In [6]:
from sklearn.metrics import (
    make_scorer,
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

scoring = {
    "roc_auc": make_scorer(roc_auc_score),
    "accuracy": make_scorer(accuracy_score),
    "precision": make_scorer(precision_score),
    "recall": make_scorer(recall_score),
    "f1": make_scorer(f1_score),
}


### LogisticRegression

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

In [93]:
params = {
    "C": 10,
    "random_state": 42,
    "verbose": 1,
    "n_jobs": -1,
}

logistic_reg = LogisticRegression(**params, tol=0.001)

# Use cross_validate with multiple scoring metrics
cv_results = cross_validate(
    logistic_reg, X_train, y_train, cv=5, scoring=scoring, n_jobs=-1, verbose=1
)

for metric in scoring.keys():
    print(f"{metric}: {cv_results[f'test_{metric}'].mean():.4f}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.


roc_auc: 0.50
accuracy: 0.92
precision: 0.46
recall: 0.01
f1: 0.02


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.9s finished


---

### RandomForestClassifier

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [33]:
# to remove if cell above is validated

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


params = {
    "n_estimators": 100,
    "random_state": 42,
    "verbose": 1,
    "n_jobs": -1,
}

random_forest_domain = RandomForestClassifier(**params)

cv_scores = cross_val_score(
    random_forest_domain, X_train, y_train, cv=cv, scoring="roc_auc"
)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   10.4s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   10.3s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    3

Cross-validation ROC-AUC scores: [0.71384326 0.72469673 0.7041607  0.71798053 0.70832107]
Mean ROC-AUC: 0.7138


[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.2s finished


In [68]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


params = {
    "n_estimators": 100,
    "random_state": 42,
    "verbose": 1,
    "n_jobs": -4,
}

random_forest_domain = RandomForestClassifier(**params)

# scoring = {
#     "roc_auc": make_scorer(roc_auc_score),
#     "accuracy": make_scorer(accuracy_score),
#     "precision": make_scorer(precision_score),
#     "recall": make_scorer(recall_score),
#     "f1": make_scorer(f1_score),
# }


cv_results = cross_validate(
    random_forest_domain,
    X_train,
    y_train,
    cv=cv,
    scoring=scoring,
    return_estimator=True,
)  # cs_results is a dictionnay

# Print average scores
print("")
for metric in ["roc_auc", "accuracy", "precision", "recall", "f1"]:
    print(f"Average {metric}: {cv_results['test_' + metric].mean():.4f}")

[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-4)]: Done 100 out of 100 | elapsed:   12.4s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-4)]: Done 100 out of 100 | elapsed:   12.6s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    4.8s
[Parall

Average roc_auc: 0.5005
Average accuracy: 0.9193
Average precision: 0.6674
Average recall: 0.0010
Average f1: 0.0020


[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.2s finished


We have **good accuracy** but **mediocre recall** (true positive/ true positives + false positives).  
THe precision is 0.62 letting us know that the model 

Our data is imbalanced, thus the model is not capturing the minority class information.

---

### SMOTE

In [10]:
from imblearn.over_sampling import SMOTE


smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check the new class distribution
print("Original class distribution:", pd.Series(y_train).value_counts())
print("Resampled class distribution:", pd.Series(y_train_resampled).value_counts())

Original class distribution: TARGET
0    282686
1     24825
Name: count, dtype: int64
Resampled class distribution: TARGET
1    282686
0    282686
Name: count, dtype: int64


---

In [11]:
# Models
models = [
    (
        "Logistic Regression",
        {"C": 10, "tol": 0.001, "random_state": 42, "verbose": 1, "n_jobs": -1},
        LogisticRegression(),
        (X_train, y_train),
    ),
    (
        "Random Forest",
        {"n_estimators": 100, "random_state": 42, "verbose": 1, "n_jobs": -4},
        RandomForestClassifier(),
        (X_train, y_train),
    ),
    (
        "Random Forest with SMOTE",
        {"n_estimators": 100, "random_state": 42, "verbose": 1, "n_jobs": -4},
        RandomForestClassifier(),
        (X_train_resampled, y_train_resampled),
    ),
]


In [14]:
cv_results_dict = {}

cv = 5

for model_name, params, model, train_data in models:
    X_train = train_data[0]
    y_train = train_data[1]

    cv_results = cross_validate(
        model.set_params(**params),
        X_train,
        y_train,
        cv=cv,
        scoring=scoring,
        return_estimator=True,
    )  # cs_results is a dictionnay

    cv_results_dict[model_name] = cv_results
    # model.set_params(**params)
    # model.fit(X_train, y_train)
    # y_pred = model.predict(X_test)
    # report = classification_report(y_test, y_pred, output_dict=True)
    # reports.append(report)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-4)]: Done 100 out of 100 | elapsed:   12.0s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-4)]: Done 100 out of 100 | el

In [16]:
def evaluate_models(models, cv=5, scoring="accuracy"):
    """
    Evaluate a list of models using cross-validation and store the results in a dictionary.

    Parameters:
    - models: List of tuples, each containing:
        - model_name: Name of the model (str)
        - params: Dictionary of parameters for the model
        - model: The model instance
        - train_data: Tuple containing (X_train, y_train)
    - cv: Number of cross-validation folds (int)
    - scoring: Scoring metric for cross-validation (str)

    Returns:
    - cv_results_dict: Dictionary containing cross-validation results for each model
    """

    # Dictionary to store results
    cv_results_dict = {}

    for model_name, params, model, train_data in models:
        X_train = train_data[0]
        y_train = train_data[1]

        cv_results = cross_validate(
            model.set_params(**params),
            X_train,
            y_train,
            cv=cv,
            scoring=scoring,
            return_estimator=True,
        )

        # Store the results in the dictionary
        cv_results_dict[model_name] = cv_results

    return cv_results_dict


cv_results_dict = {}
cv_results_dict = evaluate_models(models)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-4)]: Done 100 out of 100 | elapsed:   12.1s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-4)]: Done 100 out of 100 | el

---

# MLflow

In [24]:
import mlflow
from mlflow.models import infer_signature

In [73]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow try1")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log each metric individually
    mlflow.log_metric("accuracy", cv_results["test_accuracy"].mean())
    mlflow.log_metric("precision", cv_results["test_precision"].mean())
    mlflow.log_metric("recall", cv_results["test_recall"].mean())
    mlflow.log_metric("f1", cv_results["test_f1"].mean())

    # Log the loss metric
    # mlflow.log_metric(
    #     {
    #         #"roc_auc_score": cv_results["test_roc_auc"],
    #         "accuracy": (cv_results["test_accuracy"].mean()),
    #         "precision": (cv_results["test_precision"].mean()),
    #         "recall": (cv_results["test_recall"].mean()),
    #         "f1": (cv_results["test_f1"].mean()),
    #     }
    # )

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "RandomForest for credit classification")

    random_forest_domain.fit(X_train, y_train)

    # Infer the model signature
    signature = infer_signature(X_train, random_forest_domain.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=random_forest_domain,
        artifact_path="credit_model_test1",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

[Parallel(n_jobs=-4)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-4)]: Done  36 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-4)]: Done 100 out of 100 | elapsed:   17.4s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    1.2s finished
2025/04/07 16:04:38 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.5s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    1.3s finished
Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/04/07 16:05:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 secon

🏃 View run clumsy-koi-940 at: http://127.0.0.1:5000/#/experiments/726532937351277124/runs/2e70af58225c446989fa861ada3fd324
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/726532937351277124


---

# New model with SMOTE